In [1]:
import pandas as pd

# Load the CSV file using the full path
file_path_1 = r"C:\Users\Pranav Darekar\Documents\all_coins_ohlcv_filtered"
df_ohlcv = pd.read_csv(file_path_1)

file_path_2 = r"C:\Users\Pranav Darekar\Documents\crypto_listings_latest_sorted"
df_crypto_listings_sorted = pd.read_csv(file_path_2)


In [2]:
# Set the slug column as the index for both DataFrames
df_ohlcv.set_index('symbol', inplace=True)
df_crypto_listings_sorted.set_index('symbol', inplace=True)

# Perform an inner join on cmc_rank
df = df_crypto_listings_sorted[['cmc_rank']].join(df_ohlcv, how='inner')

# Filter rows where 'cmc_rank' is between 1 and 100 inclusive
df_raw = df[(df['cmc_rank'] >= 1) & (df['cmc_rank'] <= 100)]

In [3]:
df = df_raw
# Ensure the timestamp column is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the DataFrame by 'slug' and 'timestamp' columns
df.sort_values(by=['slug', 'timestamp'], inplace=True)

# Perform time-series calculations within each group (each cryptocurrency)
grouped = df.groupby('slug')
# Calculate percentage change for each cryptocurrency
df['m_pct_1d'] = grouped['close'].pct_change()

# Calculate cumulative returns for each cryptocurrency
df['d_pct_cum_ret'] = (1 + df['m_pct_1d']).groupby(df['slug']).cumprod() - 1

In [4]:
import pandas as pd

# Assuming df is your DataFrame and it is already sorted by 'slug' and 'timestamp'

def calculate_obv(group):
    # Initialize OBV list
    obv = [0]  # Start with zero for the first row
    for i in range(1, len(group)):
        if group['close'].iloc[i] > group['close'].iloc[i - 1]:
            obv.append(obv[-1] + group['volume'].iloc[i])
        elif group['close'].iloc[i] < group['close'].iloc[i - 1]:
            obv.append(obv[-1] - group['volume'].iloc[i])
        else:
            obv.append(obv[-1])
    return pd.Series(obv, index=group.index)

# Ensure the DataFrame has unique indices and reset if necessary
df = df.reset_index(drop=True)

# Group by 'slug' and apply OBV calculation
df['obv'] = df.groupby('slug').apply(calculate_obv).reset_index(level=0, drop=True)

# Recalculate the grouped DataFrame after adding the 'obv' column
grouped = df.groupby('slug')
df['m_tvv_obv_1d'] = grouped['obv'].pct_change()


In [5]:
# Calculate the Simple Moving Average (SMA) for 9 and 18 periods
df['SMA9'] = grouped['close'].transform(lambda x: x.rolling(window=9).mean())
df['SMA18'] = grouped['close'].transform(lambda x: x.rolling(window=18).mean())

# Calculate the Exponential Moving Average (EMA) for 9 and 18 periods
df['EMA9'] = grouped['close'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
df['EMA18'] = grouped['close'].transform(lambda x: x.ewm(span=18, adjust=False).mean())




In [6]:
# Calculate the Simple Moving Average (SMA) for 21 periods
df['SMA21'] = df.groupby('slug')['close'].transform(lambda x: x.rolling(window=21).mean())
df['SMA108'] = df.groupby('slug')['close'].transform(lambda x: x.rolling(window=108).mean())

# Calculate EMA (21-period)
df['EMA21'] = df.groupby('slug')['close'].transform(lambda x: x.ewm(span=21, adjust=False).mean())
# Calculate EMA (108-period)
df['EMA108'] = df.groupby('slug')['close'].transform(lambda x: x.ewm(span=108, adjust=False).mean())



In [7]:
def calculate_atr(group, window=14):
    # Calculate True Range
    group['prev_close'] = group['close'].shift(1)
    group['tr1'] = group['high'] - group['low']
    group['tr2'] = abs(group['high'] - group['prev_close'])
    group['tr3'] = abs(group['low'] - group['prev_close'])
    group['TR'] = group[['tr1', 'tr2', 'tr3']].max(axis=1)

    # Calculate ATR
    group['ATR'] = group['TR'].rolling(window=window).mean()

    return group

# Apply the function to each cryptocurrency
df = df.groupby('slug').apply(calculate_atr).reset_index(level=0, drop=True)


In [8]:
def calculate_keltner_channels(group, window_ema=21, window_atr=14):
    # Calculate EMA
    group['EMA21'] = group['close'].ewm(span=window_ema, adjust=False).mean()

    # Calculate ATR
    group = calculate_atr(group, window=window_atr) # calculate_atr is now defined before being called

    # Calculate Keltner Channels
    group['Keltner_Upper'] = group['EMA21'] + (group['ATR'] * 1.5)
    group['Keltner_Lower'] = group['EMA21'] - (group['ATR'] * 1.5)

    return group

# Apply the function to each cryptocurrency
df = df.groupby('slug').apply(calculate_keltner_channels).reset_index(level=0, drop=True)


In [9]:
def calculate_donchian_channels(group, window=20):
    # Calculate Donchian Channels
    group['Donchian_Upper'] = group['high'].rolling(window=window).max()
    group['Donchian_Lower'] = group['low'].rolling(window=window).min()

    return group

# Reset the index before applying the function (if needed)
df = df.reset_index(drop=True) # drop=True to avoid old index being added as a column

# Apply the function to each cryptocurrency
df = df.groupby('slug').apply(calculate_donchian_channels).reset_index(level=0, drop=True)

In [10]:


def calculate_vwap(group):
    # Calculate typical price for each period
    group['typical_price'] = (group['high'] + group['low'] + group['close']) / 3

    # Calculate the cumulative sum of typical price * volume
    group['cum_price_volume'] = (group['typical_price'] * group['volume']).cumsum()

    # Calculate the cumulative sum of volume
    group['cum_volume'] = group['volume'].cumsum()

    # Calculate VWAP
    group['VWAP'] = group['cum_price_volume'] / group['cum_volume']

    return group

# Reset the index before applying the function (if needed)
df = df.reset_index(drop=True) # drop=True to avoid old index being added as a column


# Group by 'slug' to calculate VWAP for each cryptocurrency
df = df.groupby('slug').apply(calculate_vwap).reset_index(level=0, drop=True)




In [11]:
import pandas as pd

# Correct ADL Calculation
df['ADL'] = ((df['close'] - df['low'] - (df['high'] - df['close'])) / (df['high'] - df['low'])) * df['volume']

def calculate_cmf(group, period):
    # Ensure 'slug' is not an index
    group = group.reset_index(drop=True)

    # Correct ADL Calculation
    group['ADL'] = ((group['close'] - group['low'] - (group['high'] - group['close'])) / (group['high'] - group['low'])) * group['volume']

    # Calculate cumulative ADL and volume
    group['cum_adl'] = group['ADL'].cumsum()
    group['cum_volume'] = group['volume'].cumsum()

    # Calculate CMF, handling potential division by zero
    epsilon = 1e-10  # Small constant to avoid division by zero
    group['CMF'] = group['cum_adl'].rolling(window=period).sum() / (group['cum_volume'].rolling(window=period).sum() + epsilon)

    return group

# Define the period for CMF calculation
period = 21

# Reset the index before applying the function (if needed)
df = df.reset_index(drop=True)  # drop=True to avoid old index being added as a column

# Group by 'slug' to calculate CMF for each cryptocurrency
df = df.groupby('slug').apply(calculate_cmf, period=period).reset_index(level=0, drop=True)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148093 entries, 0 to 4007
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   cmc_rank          148093 non-null  int64         
 1   id                148093 non-null  float64       
 2   slug              148093 non-null  object        
 3   name              148093 non-null  object        
 4   timestamp         148093 non-null  datetime64[ns]
 5   ref_cur_id        148093 non-null  float64       
 6   ref_cur_name      148093 non-null  object        
 7   time_open         148093 non-null  object        
 8   time_close        148093 non-null  object        
 9   time_high         148093 non-null  object        
 10  time_low          148093 non-null  object        
 11  open              148093 non-null  float64       
 12  high              148093 non-null  float64       
 13  low               148093 non-null  float64       
 14  close      

In [13]:
# @title binary

In [14]:
# prompt: name,timestamp,ref_cur_id,ref_cur_name,time_open,time_close,time_high,time_low,open,high,low,close,volume,market_cap... drop these colums from df

# List of columns to drop
columns_to_drop = ['name', 'ref_cur_id', 'ref_cur_name', 'time_open',
                   'time_close', 'time_high', 'time_low', 'open', 'high', 'low',
                   'close', 'volume', 'market_cap']

# Drop the specified columns
df_bin = df.drop(columns=columns_to_drop, errors='ignore')


In [15]:
# prompt: calculate  m_tvv_obv_1d where if the value is grater than 0 it is mapped to 1 and for value less than zero map to -1

df_bin['m_tvv_obv_1d_binary'] = df_bin['m_tvv_obv_1d'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))


In [16]:
# prompt: SMA9, SMA18, EMA9, EMA18, SMA21, SMA108, EMA21, EMA108
# mujhe crossover calculate karne hai so 9 ka 18 ke sath and 21 ka 108 ke sath hoga jab 9 18 se jyada hai toh 1 jab 9 18 se kaam hai tab -1 same jab 21 jyada hai 108 se tab 1 and jab 21 kaam hai 108 se tab -1
# mera naming conventing hai d_tvv_sma...

# Calculate crossovers for SMA9 and SMA18
df_bin['d_tvv_sma9_18'] = (df_bin['SMA9'] > df_bin['SMA18']).astype(int) * 2 - 1

# Calculate crossovers for EMA9 and EMA18
df_bin['d_tvv_ema9_18'] = (df_bin['EMA9'] > df_bin['EMA18']).astype(int) * 2 - 1

# Calculate crossovers for SMA21 and SMA108
df_bin['d_tvv_sma21_108'] = (df_bin['SMA21'] > df_bin['SMA108']).astype(int) * 2 - 1

# Calculate crossovers for EMA21 and EMA108
df_bin['d_tvv_ema21_108'] = (df_bin['EMA21'] > df_bin['EMA108']).astype(int) * 2 - 1


In [17]:
df_bin.head()

,cmc_rank,id,slug,timestamp,m_pct_1d,d_pct_cum_ret,obv,m_tvv_obv_1d,SMA9,SMA18,...,cum_volume,VWAP,ADL,cum_adl,CMF,m_tvv_obv_1d_binary,d_tvv_sma9_18,d_tvv_ema9_18,d_tvv_sma21_108,d_tvv_ema21_108
0,58,7278.0,aave,2020-10-03 23:59:59,NaN,NaN,0.00,NaN,NaN,NaN,...,0.00,NaN,0.000000,0.000000,NaN,0,-1,-1,-1,-1
1,58,7278.0,aave,2020-10-04 23:59:59,-0.008964,-0.008964,0.00,NaN,NaN,NaN,...,0.00,NaN,-0.000000,0.000000,NaN,0,-1,-1,-1,-1
2,58,7278.0,aave,2020-10-05 23:59:59,0.010331,0.001275,0.00,NaN,NaN,NaN,...,0.00,NaN,0.000000,0.000000,NaN,0,-1,-1,-1,-1
3,58,7278.0,aave,2020-10-06 23:59:59,-0.203266,-0.202250,-583091.46,-inf,NaN,NaN,...,583091.46,45.512816,-429626.168079,-429626.168079,NaN,-1,-1,-1,-1,-1
4,58,7278.0,aave,2020-10-07 23:59:59,-0.054659,-0.245854,-1265925.65,1.171058,NaN,NaN,...,1265925.65,42.262876,189752.424703,-239873.743376,NaN,1,-1,-1,-1,-1


In [18]:
# prompt: can you help me derive bullish n bearsih values for CMF ?

# Assuming 'CMF' column exists in df_bin
threshold = 0  # Adjust this threshold as needed

# Derive bullish/bearish signals based on CMF crossing the threshold
df_bin['m_tvv_cmf'] = 0  # Initialize the new column with zeros
df_bin.loc[df_bin['CMF'] > threshold, 'm_tvv_cmf'] = 1  # Bullish signal
df_bin.loc[df_bin['CMF'] < threshold, 'm_tvv_cmf'] = -1 # Bearish signal


In [19]:
df_bin.tail()

,cmc_rank,id,slug,timestamp,m_pct_1d,d_pct_cum_ret,obv,m_tvv_obv_1d,SMA9,SMA18,...,VWAP,ADL,cum_adl,CMF,m_tvv_obv_1d_binary,d_tvv_sma9_18,d_tvv_ema9_18,d_tvv_sma21_108,d_tvv_ema21_108,m_tvv_cmf
4003,7,52.0,xrp,2024-07-21 23:59:59,0.004735,100.578245,2.300163e+11,0.006518,0.569820,0.504807,...,0.680286,5.722371e+08,3.629783e+11,0.062906,1,1,1,-1,1,1
4004,7,52.0,xrp,2024-07-22 23:59:59,0.017022,102.307313,2.320684e+11,0.008922,0.578854,0.514477,...,0.680259,3.388445e+08,3.633172e+11,0.062962,1,1,1,-1,1,1
4005,7,52.0,xrp,2024-07-23 23:59:59,-0.016753,100.576558,2.304107e+11,-0.007143,0.587142,0.524031,...,0.680236,-3.047826e+08,3.630124e+11,0.063011,-1,1,1,-1,1,1
4006,7,52.0,xrp,2024-07-24 23:59:59,0.035796,104.212566,2.323438e+11,0.008390,0.596138,0.533463,...,0.680213,6.327255e+08,3.636451e+11,0.063073,1,1,1,1,1,1
4007,7,52.0,xrp,2024-07-25 23:59:59,-0.030747,100.977643,2.300369e+11,-0.009929,0.598409,0.543463,...,0.680182,-6.961188e+08,3.629490e+11,0.063140,-1,1,1,1,1,1


In [21]:
# Drop columns by their index positions
df_bin.drop(df_bin.columns[4:33], axis=1, inplace=True)


In [28]:
import pandas as pd

# Convert 'timestamp' to datetime if it's not already
df_bin['timestamp'] = pd.to_datetime(df_bin['timestamp'])

# Find the latest date in the 'timestamp' column
latest_date = df_bin['timestamp'].dt.date.max()

# Filter the DataFrame for the latest date
df_filtered = df_bin[df_bin['timestamp'].dt.date == latest_date]


In [29]:
df_oscillator_bin = df_filtered
df_oscillator = df

In [ ]:
# Export df_momentum_bin to a CSV file
df_oscillator_bin.to_csv('df_oscillator_bin.csv', index=False)
df_oscillator.to_csv('df_oscillator.csv', index=False)
